In [2]:
# Force Keras to use the CPU
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""

# NOTE: This code block silences memory allocation warnings, which are empirically annoying.
# It's a good idea to have it on regularly when you're making changes.
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

# Our imports
from keras.models import Sequential
from keras.layers import Dense
import sqlite3
import pickle
from datetime import datetime
from keras import regularizers

# old imports
from sklearn.svm import LinearSVC
from sklearn import preprocessing
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import collections

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from tensorflow import set_random_seed
seed = 24

set_random_seed(seed)
np.random.seed(seed)

In [ ]:
#encoded = np.load("./npy_data/data_encoded_d.npy")
encoded = np.load('./npy_data/train_data.npy')#, shape=(78, 25019446))

In [ ]:
blood_types = np.load('./npy_data/blood_types.npy')

In [ ]:
np.bincount(blood_types)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(encoded, blood_types, test_size=0.2)

In [11]:
X_train.shape

(63, 25019446)

In [12]:
np.bincount(y_train)

array([40, 23])

In [13]:
zeros = np.argwhere(blood_types == 0)
ones = np.argwhere(blood_types == 1)
keep_idx = np.concatenate((zeros, ones, ones))

In [14]:
keep_idx = keep_idx[:49 * 2]

In [15]:
np.random.shuffle(keep_idx)

In [16]:
blood_keep = blood_types[keep_idx]
encoded_keep = encoded[keep_idx]

In [17]:
encoded_keep.shape

(98, 1, 25019446)

In [18]:
encoded_keep = encoded_keep.ravel().reshape(-1, encoded_keep.shape[-1])

In [19]:
blood_keep = blood_keep.ravel()

In [20]:
print(encoded_keep.shape, blood_keep.shape)

((98, 25019446), (98,))


In [21]:
np.bincount(blood_keep)

array([49, 49])

In [22]:
X_train, X_test, y_train, y_test = train_test_split(encoded_keep, blood_keep, test_size=0.2)

In [23]:
y_test

array([1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0])

In [ ]:
#if model: del model
from keras import layers
model = Sequential()
model.add(Dense(10, input_shape=(encoded.shape[1],), kernel_initializer='truncated_normal'))
model.add(layers.LeakyReLU())
model.add(Dense(5, kernel_initializer='truncated_normal'))
model.add(layers.LeakyReLU())
model.add(Dense(1, kernel_initializer='truncated_normal', activation='sigmoid'))

In [ ]:
print(model.summary())

In [67]:
from keras import optimizers
model.compile(loss='binary_crossentropy', optimizer=optimizers.adam(lr=1e-1), metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, 
                     y_train, 
                     validation_data=(X_test, y_test),
                     epochs = 100, 
                     batch_size = 26,
                     verbose = 1)

In [57]:
model.predict(X_test)

array([[0.5112449],
       [0.5112449],
       [0.5112449],
       [0.5112449],
       [0.5112449],
       [0.5112449],
       [0.5112449],
       [0.5112449],
       [0.5112449],
       [0.5112449],
       [0.5112449],
       [0.5112449],
       [0.5112449],
       [0.5112449],
       [0.5112449],
       [0.5112449],
       [0.5112449],
       [0.5112449],
       [1.       ],
       [0.5112449]], dtype=float32)